In [5]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup

In [9]:
class NinisiteTopicCrawler():
    def __init__(self, topic_id):
        self.topic_id = topic_id
        self.main_link = 'https://www.ninisite.com'
        self.topic_link = self.main_link + '/discussion/topic/' + str(topic_id)
        self.topic_html = requests.get(self.topic_link).text
        self.topic_soup = BeautifulSoup(self.topic_html,'html.parser')
    
    def topic_parser(self):
        soup = self.topic_soup.find('article',{'id':'topic'})
        self.topic_detail = {
            'id' : self.topic_id,
            'title': soup.find('div',{'class':'col-xs-12 m-b-1 p-x-1 forum__topic--header'}).find('a').text,
            'text': soup.find('div',{'class':'post-message topic-post__message col-xs-12 fr-view m-b-1 p-x-1'}).text,
            'date': soup.find('span',{'class':'date'}).text,
            'username' : soup.find('a',{'class':'col-xs-9 col-md-12 text-md-center text-xs-right nickname'})['href'].split("/")[3],
            'userid' : soup.find('a',{'class':'col-xs-9 col-md-12 text-md-center text-xs-right nickname'})['href'].split("/")[2],
            'number_comments': int(soup.find_all('span',{'class':'pull-xs-right'})[-1].text.split()[1])
        }
        self.topic_detail['number_pages'] = int(np.ceil(self.topic_detail['number_comments']/20))
    
  
    def comment_parser(self, comment):
        self.comment = comment
        comment_detail = {
             'id' : comment['id'].split('-')[-1],
             'text': comment.find('div', class_='post-message').text.strip() ,
             'date': comment.find('span', class_='date').text.strip(),
             'username' : comment.find('a', class_='col-xs-9 col-md-12 text-md-center text-xs-right nickname').text.strip(),
             'userid' : comment.find('a', class_='col-xs-9 col-md-12 text-md-center text-xs-right nickname')['href'].split('/')[2],
             'isReply' : bool(comment.find('div', {'class':'reply-message'}))
        }
        return comment_detail
    
    
    def crawl_comments_in_topic(self):
        self.topic_parser()
        comments_data = []
        for pg_number in range(1,self.topic_detail['number_pages']+1):
            print(pg_number)
            html = requests.get(self.topic_link+f'?page={pg_number}').text
            soup = BeautifulSoup(html,'html.parser')
            self.comments = soup.find_all('article', class_='topic-post')
            for comment in self.comments[1:]:
                comments_data.append(self.comment_parser(comment))
            
            time.sleep(1.5)
                
        return pd.DataFrame(comments_data)
        
    

In [10]:
crawler = NinisiteTopicCrawler(11158288)

In [11]:
crawler.crawl_comments_in_topic()

1
2
3
4


,id,text,date,username,userid,isReply
0,292336470,شوهر ندارم پس خودمو,1402/03/03,توت_فرنگی_م,cb23f48c-166e-4641-94d5-7a58cc4b7ec1,False
1,292336504,شوهرم,1402/03/03,goldgirl_,c464cc09-7454-443c-9746-3181b5ed5dc5,False
2,292336542,معلومه که خودم,1402/03/03,ماهور57,b5353247-d275-419c-b7fc-1ffe368a29b5,False
3,292336554,خودمو,1402/03/03,چیکووووو,929e293f-1fd2-410d-afe4-1c65d21801ee,False
4,292336559,شوهرم و بچه هام,1402/03/03,herng,afaf28ac-8d8d-40c1-9759-10f5cad3d16e,False
...,...,...,...,...,...,...
72,292347045,قبلنا اول شوهر و بچه و خودم دیدم من تو زندگیش ...,1402/03/03,نازش,e9d5055b-71e3-47ea-8520-5de2842e5495,False
73,292347121,😘🌹نه ممنونم تو پست قبلی گفتم موضوع چیه دیگه با...,1402/03/03,شهلا_,741e7a29-763c-4b7e-8d52-c43392a5199d,True
74,292347279,نه بابا امتحان کنم که چی !!آخه مگه من مفتش هست...,1402/03/03,شهلا_,741e7a29-763c-4b7e-8d52-c43392a5199d,True
75,292347456,"Прости, я осудил тебя, не зная. Удачи, моя дор...",1402/03/03,توراکینا,41afaebc-d858-40cf-97cf-689bc06e2eef,True
